In [ ]:
# Improve by ChatGPT & GitHub Copilot
# Running the requierements.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/OSSE_generator/requierements.ipynb

# Loading files

## DAC

In [ ]:
dac_cropped_1h = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/DAC_limpio/dac_cropped_1h.nc").load() 

In [ ]:
dac_cropped_1h = dac_cropped_1h.rename({
    'time': 'time_counter',
    'longitude': 'nav_lon',
    'latitude': 'nav_lat'
})

In [ ]:
dac_cropped_1h

## SSH

In [ ]:
# Loading SSH
ssh = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60/*_SSH.nc", combine="nested", concat_dim="time_counter", parallel=True)

In [ ]:
import xesmf as xe

# Define the source grid from `nav_lat` and `nav_lon`
source_grid = {'lon': ssh['nav_lon'].values, 'lat': ssh['nav_lat'].values}
# Define the target grid from `dac_cropped` dataset
target_grid = {'lon': dac_cropped_1h['nav_lon'].values, 'lat': dac_cropped_1h['nav_lat'].values}
# Create a regridder object
regridder = xe.Regridder(source_grid, target_grid, 'bilinear')

# Perform the regridding
ssh_dacgrid = regridder(ssh['zos'])


In [ ]:
ssh_dacgrid = ssh_dacgrid.drop_vars(['time_instant'])
ssh_dacgrid = ssh_dacgrid.rename({
    'lon': 'nav_lon',
    'lat': 'nav_lat'
})

In [ ]:
ssh_dacgrid.to_netcdf("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60/ssh_dacgrid.nc")

In [ ]:
ssh_dacgrid = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60/ssh_dacgrid.nc")

In [ ]:
ssh_dacgrid.load() 

#tarda como 20 min

## SSH - DAC

In [ ]:
sshdedac = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60_dedac/ssh_dedac_masked_*.nc", combine="nested", concat_dim="time_counter", parallel=True)

In [ ]:
import xesmf as xe

# Define the source grid from `nav_lat` and `nav_lon`
source_grid = {'lon': sshdedac['nav_lon'].values, 'lat': sshdedac['nav_lat'].values}
# Define the target grid from `dac_cropped` dataset
target_grid = {'lon': dac_cropped_1h['nav_lon'].values, 'lat': dac_cropped_1h['nav_lat'].values}
# Create a regridder object
regridder = xe.Regridder(source_grid, target_grid, 'bilinear')

# Perform the regridding
sshdedac_dacgrid = regridder(sshdedac['ssh_dedac'])

In [ ]:
sshdedac_dacgrid.to_netcdf("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60_dedac/sshdedac_dacgrid.nc")

In [ ]:
sshdedac_dacgrid = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60_dedac/sshdedac_dacgrid.nc")

In [ ]:
sshdedac_dacgrid.load() 

# Covarianza

## cov(SSH,DAC)

In [ ]:
# Extracting the 'dac' DataArray from the 'dac_cropped_1h' Dataset
dac_data_array = dac_cropped_1h['dac'].load()

In [ ]:
# Assuming 'ssh_dacgrid' is already a DataArray, you can now calculate the covariance
cov1 = xr.cov(dac_data_array, ssh_dacgrid, dim='time_counter')

In [ ]:
cov1

In [ ]:
# # Define your custom colormap and normalization
# cmap_colors = [(0.294, 0.0, 0.510), (1, 1, 1), (0.698, 0.133, 0.133)]  # Purple, White, Red
# custom_cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', cmap_colors)
# custom_cmap.set_bad(color='black')  # Uncomment and adjust if you want to handle NaN values specifically
# norm = mcolors.TwoSlopeNorm(vmin=-0.005, vcenter=0, vmax=0.007)

plt.figure(
    #figsize=(12, 10)  # Uncomment and adjust if you want to specify the figure size
)
# Create a plot and get the axis object, setting the projection to PlateCarree
ax = plt.axes(projection=ccrs.PlateCarree())

# Here, specify the desired minimum and maximum values for your color scale
min_val = -0.005  # Example minimum value
max_val = 0.007  # Example maximum value

# Use pcolormesh with the axis object
pcolormesh = ax.pcolormesh(cov1['nav_lon'], cov1['nav_lat'], cov1,cmap='coolwarm',
                           vmin=min_val, vmax=max_val
                           )
# Add colorbar and label it
cbar = plt.colorbar(pcolormesh, ax=ax)
cbar.set_label('Variance (m²)')

# Set labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('cov(SSH, DAC)')

# Add land with higher resolution to the main plot on top, filled black
ax.add_feature(cfeature.LAND.with_scale('10m'), edgecolor='black', facecolor='black', zorder=3)

# Add gridlines with labels to the main plot
gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.2, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 9, 'color': 'black'}
gl.ylabel_style = {'size': 9, 'color': 'black'}

# Fix the aspect ratio
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

In [ ]:
var_ssh = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/varianzas/var_ssh.nc")

In [ ]:
# Perform interpolation for the current month
cov1_sshgrid = cov1.interp(
    nav_lat=var_ssh.nav_lat,
    nav_lon=var_ssh.nav_lon,
    method='linear'
    )

In [ ]:
cov1_sshgrid

In [ ]:
# Define your custom colormap and normalization
cmap_colors = [(0.294, 0.0, 0.510), (1, 1, 1), (0.698, 0.133, 0.133)]  # Purple, White, Red
custom_cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', cmap_colors)
custom_cmap.set_bad(color='black')  # Uncomment and adjust if you want to handle NaN values specifically
norm = mcolors.TwoSlopeNorm(vmin=-0.005, vcenter=0, vmax=0.007)

plt.figure(
    #figsize=(12, 10)  # Uncomment and adjust if you want to specify the figure size
)
# Create a plot and get the axis object, setting the projection to PlateCarree
ax = plt.axes(projection=ccrs.PlateCarree())

# Here, specify the desired minimum and maximum values for your color scale
min_val = -0.005  # Example minimum value
max_val = 0.007  # Example maximum value

# Use pcolormesh with the axis object
pcolormesh = ax.pcolormesh(cov1_sshgrid['nav_lon'], cov1_sshgrid['nav_lat'], cov1_sshgrid, cmap=custom_cmap, norm=norm,
                           #vmin=min_val, vmax=max_val
                           )
# Add colorbar and label it
cbar = plt.colorbar(pcolormesh, ax=ax)
cbar.set_label('Covariance (m²)')

# Set labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('cov(SSH, DAC)')

# Add land with higher resolution to the main plot on top, filled black
ax.add_feature(cfeature.LAND.with_scale('10m'), edgecolor='black', facecolor='black', zorder=3)

# Add gridlines with labels to the main plot
gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.2, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 9, 'color': 'black'}
gl.ylabel_style = {'size': 9, 'color': 'black'}

# Fix the aspect ratio
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

In [ ]:
var_dac = dac_cropped_1h.var(dim='time_counter', skipna=True)

In [ ]:
# # Define your custom colormap and normalization
# cmap_colors = [(0.294, 0.0, 0.510), (1, 1, 1), (0.698, 0.133, 0.133)]  # Purple, White, Red
# custom_cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', cmap_colors)
# custom_cmap.set_bad(color='black')  # Uncomment and adjust if you want to handle NaN values specifically
# norm = mcolors.TwoSlopeNorm(vmin=-0.005, vcenter=0, vmax=0.007)

plt.figure(
    #figsize=(12, 10)  # Uncomment and adjust if you want to specify the figure size
)
# Create a plot and get the axis object, setting the projection to PlateCarree
ax = plt.axes(projection=ccrs.PlateCarree())

# Here, specify the desired minimum and maximum values for your color scale
# min_val = 0  # Example minimum value
# max_val = 0.25 # Example maximum value

# Define your custom colormap and normalization
cmap_colors = [(0.294, 0.0, 0.510), (1, 1, 1), (0.698, 0.133, 0.133)]  # Purple, White, Red
custom_cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', cmap_colors)
#custom_cmap.set_bad(color='black')  # Uncomment and adjust if you want to handle NaN values specifically
norm = mcolors.TwoSlopeNorm(vmin=-0.005, vcenter=0, vmax=0.007)

min_val = -0.005  # Example minimum value
max_val = 0.007  # Example maximum value

# # Use pcolormesh with the axis object

# pcolormesh = ax.pcolormesh(var_dac['nav_lon'], var_dac['nav_lat'], var_dac['dac'],cmap='Reds',
#                            vmin=min_val, vmax=max_val
#                            )

pcolormesh = ax.pcolormesh(var_dac['nav_lon'], var_dac['nav_lat'], var_dac['dac'],cmap=custom_cmap,
                           vmin=min_val, vmax=max_val
                           )
# Add colorbar and label it
cbar = plt.colorbar(pcolormesh, ax=ax)
cbar.set_label('Variance (m²)')

# Set labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('var(DAC)')

# Add land with higher resolution to the main plot on top, filled black
ax.add_feature(cfeature.LAND.with_scale('10m'), edgecolor='black', facecolor='black', zorder=3)

# Add gridlines with labels to the main plot
gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.2, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 9, 'color': 'black'}
gl.ylabel_style = {'size': 9, 'color': 'black'}

# Fix the aspect ratio
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

## cov(SSHdedac,DAC)

In [ ]:
# Extracting the 'dac' DataArray from the 'dac_cropped_1h' Dataset
dac_data_array = dac_cropped_1h['dac'].load()

In [ ]:
dac_data_array

In [ ]:
# Extracting the 'dac' DataArray from the 'dac_cropped_1h' Dataset
sshdedac_dacgrid_data_array = sshdedac_dacgrid['__xarray_dataarray_variable__'].load()

sshdedac_dacgrid_data_array = sshdedac_dacgrid_data_array.rename({
    'lon': 'nav_lon',
    'lat': 'nav_lat'
})

In [ ]:
sshdedac_dacgrid_data_array

In [ ]:
# Assuming 'ssh_dacgrid' is already a DataArray, you can now calculate the covariance
cov2 = xr.cov(dac_data_array, sshdedac_dacgrid_data_array, dim='time_counter')

In [ ]:
# Perform interpolation for the current month
cov2_sshgrid = cov2.interp(
    nav_lat=var_ssh.nav_lat,
    nav_lon=var_ssh.nav_lon,
    method='linear'
    )

In [ ]:
# Define your custom colormap and normalization
cmap_colors = [(0.294, 0.0, 0.510), (1, 1, 1), (0.698, 0.133, 0.133)]  # Purple, White, Red
custom_cmap = mcolors.LinearSegmentedColormap.from_list('custom_cmap', cmap_colors)
#custom_cmap.set_bad(color='black')  # Uncomment and adjust if you want to handle NaN values specifically
norm = mcolors.TwoSlopeNorm(vmin=-0.005, vcenter=0, vmax=0.007)

plt.figure(
    #figsize=(12, 10)  # Uncomment and adjust if you want to specify the figure size
)
# Create a plot and get the axis object, setting the projection to PlateCarree
ax = plt.axes(projection=ccrs.PlateCarree())

# Here, specify the desired minimum and maximum values for your color scale
# min_val = -0.005  # Example minimum value
# max_val = 0.007  # Example maximum value

# Here, specify the desired minimum and maximum values for your color scale
min_val = 0.00  # Example minimum value
max_val = 0.25  # Example maximum value

# Use pcolormesh with the axis object
pcolormesh = ax.pcolormesh(cov2_sshgrid['nav_lon'], cov2_sshgrid['nav_lat'], cov2_sshgrid, cmap=custom_cmap, norm=norm,
                           #vmin=min_val, vmax=max_val
                           )
# Add colorbar and label it
cbar = plt.colorbar(pcolormesh, ax=ax)
cbar.set_label('Covariance (m²)')

# Set labels and title
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.set_title('cov(SSH - DAC, DAC)')

# Add land with higher resolution to the main plot on top, filled black
ax.add_feature(cfeature.LAND.with_scale('10m'), edgecolor='black', facecolor='black', zorder=3)

# Add gridlines with labels to the main plot
gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.2, linestyle='-')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 9, 'color': 'black'}
gl.ylabel_style = {'size': 9, 'color': 'black'}

# Fix the aspect ratio
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()